In [1]:
import numpy as np
from numpy import ndarray
from typing import Callable, Dict, Tuple, List

In [2]:
# Linear Regression
def forward_linear_regression(X_batch: ndarray, y_batch: ndarray, weights: Dict[str, ndarray]) -> Tuple[float, Dict[str, ndarray]]:
    assert X_batch.shape[0] == y_batch.shape[0], "X and y batches number of rows do not match"
    assert X_batch.shape[1] == weights["W"].shape[0], "X batch number of columns and weights number of rows do not match"
    assert weights["B"].shape[0] == weights["W"].shape[1] == 1, "B number of rows and W number of columns different from 1"

    N = np.dot(X_batch, weights["W"])
    P = N + weights["B"]
    loss = np.mean(np.power(y_batch - P, 2))

    # save the information computed on the forward pass
    forward_info = {
        "X": X_batch,
        "N": N,
        "P": P,
        "y": y_batch,
    }

    return loss, forward_info

X_batch = np.array([
    [0, 1, 2], # X1
    [3, 4, 5], # X2
    [5, 6, 7], # X3
])

weights = {
    # "B": np.array([[2], [2], [2]]),
    "B": np.array([[2]]), # will be broadcasted (?) as if np.array([[2], [2], [2]])
    # "B": np.array([2]), # will be broadcasted (?) as if np.array([[2], [2], [2]])
    "W": np.array([
        # W1
        [1], 
        [2], 
        [3],
    ]),
}

y_batch = np.array([
    [11],
    [27],
    [41],
])

loss, forward_info = forward_linear_regression(X_batch, y_batch, weights)
loss, forward_info

(1.0,
 {'X': array([[0, 1, 2],
         [3, 4, 5],
         [5, 6, 7]]),
  'N': array([[ 8],
         [26],
         [38]]),
  'P': array([[10],
         [28],
         [40]]),
  'y': array([[11],
         [27],
         [41]])})

In [3]:
# Calculating the Gradients
def loss_gradients(forward_info: Dict[str, ndarray], weights: Dict[str, ndarray]) -> Dict[str, ndarray]:
    dLdP = -2 * (forward_info["y"] - forward_info["P"])
    dPdN = np.ones_like(forward_info["N"])
    dLdN = dLdP * dPdN
    dNdW = np.transpose(forward_info["X"])
    dLdW = np.dot(dNdW, dLdN)

    dPdB = np.ones_like(weights["B"])
    dLdB = dLdP * dPdB
    B = dLdB.sum(axis=0) # axis sum due to 1-dim B (?)

    # intermdiate results added to B and W for debugging purposes
    loss_gradients = {
        "dLdP": dLdP,
        "dPdN": dPdN,
        "dLdN": dLdN,
        "dNdW": dNdW,
        "W": dLdW,
        "dPdB": dPdB,
        "dLdB": dLdB,
        "B": B, 
    }
    return loss_gradients

loss_gradients(forward_info, weights)

{'dLdP': array([[-2],
        [ 2],
        [-2]]),
 'dPdN': array([[1],
        [1],
        [1]]),
 'dLdN': array([[-2],
        [ 2],
        [-2]]),
 'dNdW': array([[0, 3, 5],
        [1, 4, 6],
        [2, 5, 7]]),
 'W': array([[-4],
        [-6],
        [-8]]),
 'dPdB': array([[1]]),
 'dLdB': array([[-2],
        [ 2],
        [-2]]),
 'B': array([-2])}

In [4]:
# Using These Gradients to Train the Model
def init_weights(n_in: int) -> Dict[str, ndarray]:
    weights = {
    "W": np.random.randn(n_in, 1),
    "B": np.random.randn(1, 1),
    }
    return weights

init_weights(3)   

{'W': array([[ 0.28766053],
        [ 1.31596113],
        [-0.53786283]]),
 'B': array([[-0.38904971]])}

In [5]:
def permute_data(X: ndarray, y: ndarray):
    perm = np.random.permutation(X.shape[0])
    return X[perm], y[perm], perm

X = np.array([
    [0, 1, 2], # X1
    [3, 4, 5], # X2
    [5, 6, 7], # X3
])

y = np.array([
    [11],
    [27],
    [41],
])

permute_data(X, y)

(array([[0, 1, 2],
        [3, 4, 5],
        [5, 6, 7]]),
 array([[11],
        [27],
        [41]]),
 array([0, 1, 2]))

In [6]:
Batch = Tuple[ndarray, ndarray]

def generate_batch(X: ndarray, y: ndarray, start: int = 0, batch_size: int = 10) -> Batch:
    assert X.ndim == y.ndim == 2, "X and Y must be 2 dimensional"
    if start+batch_size > X.shape[0]:
        batch_size = X.shape[0] - start    
    X_batch, y_batch = X[start:start+batch_size], y[start:start+batch_size]    
    return X_batch, y_batch
    
X = np.array([
    [0, 1, 2], # X1
    [3, 4, 5], # X2
    [5, 6, 7], # X3
])

y = np.array([
    [11],
    [27],
    [41],
])

generate_batch(X, y, 0, 2)

(array([[0, 1, 2],
        [3, 4, 5]]),
 array([[11],
        [27]]))

In [7]:
from sklearn.datasets import load_boston
boston = load_boston()
data = boston.data
target = boston.target
features = boston.feature_names
data[0:3], target[0:3], features[0:3]

(array([[6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
         6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02,
         1.5300e+01, 3.9690e+02, 4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         6.4210e+00, 7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
         1.7800e+01, 3.9690e+02, 9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         7.1850e+00, 6.1100e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
         1.7800e+01, 3.9283e+02, 4.0300e+00]]),
 array([24. , 21.6, 34.7]),
 array(['CRIM', 'ZN', 'INDUS'], dtype='<U7'))

In [8]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)
data[0:3]

array([[-0.41978194,  0.28482986, -1.2879095 , -0.27259857, -0.14421743,
         0.41367189, -0.12001342,  0.1402136 , -0.98284286, -0.66660821,
        -1.45900038,  0.44105193, -1.0755623 ],
       [-0.41733926, -0.48772236, -0.59338101, -0.27259857, -0.74026221,
         0.19427445,  0.36716642,  0.55715988, -0.8678825 , -0.98732948,
        -0.30309415,  0.44105193, -0.49243937],
       [-0.41734159, -0.48772236, -0.59338101, -0.27259857, -0.74026221,
         1.28271368, -0.26581176,  0.55715988, -0.8678825 , -0.98732948,
        -0.30309415,  0.39642699, -1.2087274 ]])

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)
X_train[0:3], X_test[0:3], y_train[0:3], y_test[0:3]

(array([[-0.41248185, -0.48772236, -1.30687771, -0.27259857, -0.83528384,
          1.22857665, -0.51117971,  1.07773662, -0.75292215, -1.10611514,
          0.1130321 ,  0.44105193, -1.02650148],
        [ 5.53032093, -0.48772236,  1.01599907, -0.27259857,  0.36544404,
         -0.75169891,  1.11749449, -1.13234958,  1.66124525,  1.53092646,
          0.80657583, -3.88219457, -0.3564708 ],
        [ 9.9339306 , -0.48772236,  1.01599907, -0.27259857,  1.00468047,
          0.97356275,  0.82945385, -1.13068579,  1.66124525,  1.53092646,
          0.80657583,  0.44105193,  0.63876307]]),
 array([[-0.41643272,  3.5896366 , -1.23392304, -0.27259857, -1.19722987,
          2.23438568, -1.25795172,  0.62889308, -0.6379618 , -1.09423658,
         -1.73641788,  0.39587878, -1.23956563],
        [ 1.65868822, -0.48772236,  1.01599907, -0.27259857,  1.00468047,
         -0.08780796,  1.11749449, -1.14513701,  1.66124525,  1.53092646,
          0.80657583,  0.40640456,  1.27935726],
        [ 0.2

In [10]:
y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)
y_train[0:3], y_test[0:3]

(array([[36.2],
        [15. ],
        [10.4]]),
 array([[48.5],
        [10.2],
        [15.2]]))

In [11]:
def train(X: ndarray, 
          y: ndarray, 
          n_iter: int = 1000,
          learning_rate: float = 0.001,
          batch_size: int = 100,
          return_losses: bool = True, 
          return_weights: bool = True, 
          seed: int = 1) -> None:
    if seed:
        np.random.seed(seed)
    start = 0
    weights = init_weights(X.shape[1])
    X, y, dummy = permute_data(X, y)
    losses = []
    for i in range(n_iter):
        if start >= X.shape[0]:
            X, y, dummy = permute_data(X, y)
            start = 0
        X_batch, y_batch = generate_batch(X, y, start, batch_size)
        start += batch_size
        loss, forward_info = forward_linear_regression(X_batch, y_batch, weights)
        if return_losses:
            losses.append(loss)
        loss_grads = loss_gradients(forward_info, weights)
        for key in weights.keys():
            weights[key] -= learning_rate * loss_grads[key]
    
    if return_weights:
        return losses, weights

train_info = train(X_train, y_train, n_iter = 1000, batch_size=23, seed=180708)
losses, weights = train_info
losses[0:3], "...", losses[-3:], weights

([813.2098258057695, 707.3600829629216, 419.780274960034],
 '...',
 [36.717795915470546, 14.400552796770944, 12.46628339805241],
 {'W': array([[-1.09542534],
         [ 0.73529155],
         [ 0.19771754],
         [ 0.74750455],
         [-2.28494893],
         [ 2.20059621],
         [ 0.7194554 ],
         [-2.6384484 ],
         [ 2.53576919],
         [-1.68915096],
         [-2.31624663],
         [ 0.84352868],
         [-4.25526828]]),
  'B': array([[22.63115155]])})